In [1]:
spark.sparkContext.setLogLevel('WARN')

In [2]:
df = spark.read.text('../alice-in-wonderland.txt')
df.createOrReplaceTempView('book')
query = open('./sparkhelloworld/query.sql').read()

In [10]:
print(query)

with words as (
    select explode(split(lcase(regexp_replace(value, '[^a-zA-Z ]', '')), ' ')) as word 
    from book
)
select word, count(*) as ct
from words
where len(word) > 0
group by word
order by ct desc


In [17]:
from pyspark.sql.functions import *
words = df.select(explode(split(lower(regexp_replace("value", '[^a-zA-Z ]', '')), ' ')).alias('word'))
words.show()

+----------+
|      word|
+----------+
|    alices|
|adventures|
|        in|
|wonderland|
|          |
|        by|
|     lewis|
|   carroll|
|          |
|       the|
|millennium|
|   fulcrum|
|   edition|
|          |
|          |
|  contents|
|          |
|          |
|   chapter|
|         i|
+----------+
only showing top 20 rows



In [23]:
counts = words.where(length("word") > 0).groupBy("word").count().orderBy(desc("count"))
counts.show()

+-----+-----+
| word|count|
+-----+-----+
|  the| 1815|
|  and|  911|
|   to|  802|
|    a|  690|
|   of|  626|
|   it|  538|
|  she|  538|
| said|  462|
|  you|  430|
|   in|  427|
|    i|  401|
|alice|  385|
|  was|  359|
| that|  291|
|   as|  272|
|  her|  248|
| with|  226|
|   at|  220|
|   on|  204|
|  all|  197|
+-----+-----+
only showing top 20 rows



In [2]:
import boto3
import pandas as pd

# Read CSV file into a DataFrame
schema = {
    'customer_id': str,
    'username': str,
    'name': str,
    'sex': str,
    'email': str,
    'birthdate': str
}
df = pd.read_csv('./customer_ids_hbase.txt', delimiter='\t',dtype=schema)
print(df.head(10))
print(df.dtypes)

  customer_id        username              name sex  \
0    10000001  allenpriscilla  Margaret Johnson   F   
1    10000017         kerri75     Aaron Johnson   M   
2    10000018     stewartlisa    Madison Mathis   F   
3    10000023       crystal71       Lynn Chavez   F   
4    10000044  nicholascherry     Andrew Berger   M   
5    10000055         susan36    Daniel Johnson   M   
6    10000081         janet02    Rachel Wilkins   F   
7    10000113     laratristan    Tammy Williams   F   
8    10000121     millerwendy      Kelly Nelson   F   
9    10000166     kennethcarr    Andrew Barrera   M   

                          email   birthdate  
0            dmeadows@yahoo.com  1991-08-01  
1            ykrueger@yahoo.com  1942-01-07  
2           maureen76@gmail.com  2003-03-21  
3            philip67@gmail.com  1990-01-05  
4       kevinnorman@hotmail.com  1979-05-14  
5           zlittle@hotmail.com  1910-05-21  
6         jordanana@hotmail.com  1939-07-20  
7             david11@gmai

In [4]:
customer_ids = df['customer_id'].head(10)

In [ ]:
import boto3

my_session = boto3.Session(profile_name='hwe_prod')

# Create a DynamoDB client
dynamodb = my_session.client('dynamodb')

In [10]:
def get_dynamo_key(id):
    return { 'customer_id': {'S': id} }

keys = [get_dynamo_key(cust_id) for cust_id in customer_ids]
request_items = {
    'customers': {
        'Keys': keys
    }
}
result = dynamodb.batch_get_item(RequestItems=request_items)
print(result['Responses']['customers'])

[{'birthdate': {'S': '1918-12-10'}, 'sex': {'S': 'F'}, 'customer_id': {'S': '10000121'}, 'username': {'S': 'millerwendy'}, 'email': {'S': 'victoriamartinez@hotmail.com'}, 'name': {'S': 'Kelly Nelson'}}, {'birthdate': {'S': '1927-10-09'}, 'sex': {'S': 'M'}, 'customer_id': {'S': '10000166'}, 'username': {'S': 'kennethcarr'}, 'email': {'S': 'zimmermanerica@gmail.com'}, 'name': {'S': 'Andrew Barrera'}}, {'birthdate': {'S': '1990-01-05'}, 'sex': {'S': 'F'}, 'customer_id': {'S': '10000023'}, 'username': {'S': 'crystal71'}, 'email': {'S': 'philip67@gmail.com'}, 'name': {'S': 'Lynn Chavez'}}, {'birthdate': {'S': '1910-05-21'}, 'sex': {'S': 'M'}, 'customer_id': {'S': '10000055'}, 'username': {'S': 'susan36'}, 'email': {'S': 'zlittle@hotmail.com'}, 'name': {'S': 'Daniel Johnson'}}, {'birthdate': {'S': '1991-08-01'}, 'sex': {'S': 'F'}, 'customer_id': {'S': '10000001'}, 'username': {'S': 'allenpriscilla'}, 'email': {'S': 'dmeadows@yahoo.com'}, 'name': {'S': 'Margaret Johnson'}}, {'birthdate': {'S'

In [13]:
data = [(item['customer_id']['S'], item['username']['S'], item['name']['S'], item['email']['S'], item['birthdate']['S']) for item in result['Responses']['customers']]
print(data)

[('10000121', 'millerwendy', 'Kelly Nelson', 'victoriamartinez@hotmail.com', '1918-12-10'), ('10000166', 'kennethcarr', 'Andrew Barrera', 'zimmermanerica@gmail.com', '1927-10-09'), ('10000023', 'crystal71', 'Lynn Chavez', 'philip67@gmail.com', '1990-01-05'), ('10000055', 'susan36', 'Daniel Johnson', 'zlittle@hotmail.com', '1910-05-21'), ('10000001', 'allenpriscilla', 'Margaret Johnson', 'dmeadows@yahoo.com', '1991-08-01'), ('10000113', 'laratristan', 'Tammy Williams', 'david11@gmail.com', '1925-04-16'), ('10000081', 'janet02', 'Rachel Wilkins', 'jordanana@hotmail.com', '1939-07-20'), ('10000017', 'kerri75', 'Aaron Johnson', 'ykrueger@yahoo.com', '1942-01-07'), ('10000018', 'stewartlisa', 'Madison Mathis', 'maureen76@gmail.com', '2003-03-21'), ('10000044', 'nicholascherry', 'Andrew Berger', 'kevinnorman@hotmail.com', '1979-05-14')]


In [16]:
customerDf = spark.createDataFrame(data, ['customer_id', 'username', 'name', 'email', 'birthdate'])

In [17]:
customerDf.show()

+-----------+--------------+----------------+--------------------+----------+
|customer_id|      username|            name|               email| birthdate|
+-----------+--------------+----------------+--------------------+----------+
|   10000121|   millerwendy|    Kelly Nelson|victoriamartinez@...|1918-12-10|
|   10000166|   kennethcarr|  Andrew Barrera|zimmermanerica@gm...|1927-10-09|
|   10000023|     crystal71|     Lynn Chavez|  philip67@gmail.com|1990-01-05|
|   10000055|       susan36|  Daniel Johnson| zlittle@hotmail.com|1910-05-21|
|   10000001|allenpriscilla|Margaret Johnson|  dmeadows@yahoo.com|1991-08-01|
|   10000113|   laratristan|  Tammy Williams|   david11@gmail.com|1925-04-16|
|   10000081|       janet02|  Rachel Wilkins|jordanana@hotmail...|1939-07-20|
|   10000017|       kerri75|   Aaron Johnson|  ykrueger@yahoo.com|1942-01-07|
|   10000018|   stewartlisa|  Madison Mathis| maureen76@gmail.com|2003-03-21|
|   10000044|nicholascherry|   Andrew Berger|kevinnorman@hotma..

In [30]:
reviews_df = spark.read.option('delimiter', '\t').option("header", "true").csv('./amazon_reviews_us_Toys_v1_20.tsv')
reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [43]:
def get_dynamo_key(id):
    return { 'customer_id': {'S': id} }

def dynamo_item_to_tuple(item):
    return (item['customer_id']['S'], item['username']['S'], item['name']['S'], item['email']['S'], item['birthdate']['S'])

def get_customer_ids(reviews_df):
    return [c.customer_id for c in reviews_df.select('customer_id').collect()]

def get_customers_from_dynamo(customer_ids):
    customer_id_keys = [get_dynamo_key(cid) for cid in customer_ids]
    request_items = {
        'customers': {
            'Keys': customer_id_keys
        }
    }
    return dynamodb.batch_get_item(RequestItems=request_items)

def dynamo_response_to_dataframe(dynamo_result):
    data = [dynamo_item_to_tuple(item) for item in dynamo_result['Responses']['customers']]
    return spark.createDataFrame(data, ['customer_id', 'username', 'name', 'email', 'birthdate'])

In [44]:
# first, get all of the customer IDs from each review
customer_ids = get_customer_ids(reviews_df)
# second, using those IDs, retrieve each customer record from dynamo
dynamo_result = get_customers_from_dynamo(customer_ids)
# finally, create a new "customers" dataframe using that data
customers_df = dynamo_response_to_dataframe(dynamo_result)
customers_df.show()

+-----------+---------------+------------------+--------------------+----------+
|customer_id|       username|              name|               email| birthdate|
+-----------+---------------+------------------+--------------------+----------+
|   18778586|          nsoto|  Jill Dickson PhD|collinsalexa@hotm...|1950-10-22|
|   13545982|brendanfranklin|    James Jacobson|jennifer72@gmail.com|1915-02-07|
|   27225859|     jennifer33|    Sheri Mitchell|williamswalter@gm...|1994-01-18|
|   18461411|    davislauren|  Kathryn Martinez|blakemaxwell@gmai...|1923-05-05|
|    1297934|      velezjoel|    Kevin Robinson|racheljohnson@hot...|1969-09-30|
|    2749569|    sarahmiller| Debbie Herrera MD|  john38@hotmail.com|1977-06-23|
|    1662075|    sherryallen|       Roger Dixon| heather62@yahoo.com|1910-04-03|
|   11613707|   jerrypadilla|     James Mcclain|brownjacqueline@g...|2003-07-21|
|   23310293|        chris47|     Matthew Evans|davidwells@hotmai...|2014-04-23|
|   13394189|     jonathan57

In [49]:
joined_df = reviews_df.join(customers_df, 'customer_id')
joined_df.select('customer_id', 'review_id', 'product_title', 'username', 'name').show()

+-----------+--------------+--------------------+---------------+------------------+
|customer_id|     review_id|       product_title|       username|              name|
+-----------+--------------+--------------------+---------------+------------------+
|   18778586| RDIJS7QYB6XNR|Monopoly Junior B...|          nsoto|  Jill Dickson PhD|
|   13545982|R1T96CG98BBA15|Team Losi 8IGHT-E...|brendanfranklin|    James Jacobson|
|   27225859| R4R337CCDWLNG|Franklin Sports M...|     jennifer33|    Sheri Mitchell|
|   18461411|R2SDXLTLF92O0H|Teenage Mutant Ni...|    davislauren|  Kathryn Martinez|
|    1297934|R1CB783I7B0U52|Claw Climber Goli...|      velezjoel|    Kevin Robinson|
|    2749569|R3SORMPJZO3F2J|Big Bang Cosmic P...|    sarahmiller| Debbie Herrera MD|
|    1662075|R1YS3DS218NNMD|ZuZo 2.4GHz 4 CH ...|    sherryallen|       Roger Dixon|
|   11613707| RSUHRJFJIRB3Z|Disney Baby: Eeyo...|   jerrypadilla|     James Mcclain|
|   23310293|R298788GS6I901|Barbie Doll and F...|        chris47|

In [58]:
from pyspark.sql.functions import from_csv
reviews_df = spark.read.text('./amazon_reviews_us_Toys_v1_20.tsv')
schema = 'marketplace STRING, customer_id STRING, review_id STRING, product_id STRING, product_parent STRING, product_title STRING, product_category STRING, star_rating STRING, helpful_votes STRING, total_votes STRING, vine STRING, verified_purchase STRING, review_headline STRING, review_body STRING, review_date STRING'
reviews_df.select(from_csv('value', schema, { 'delimiter': '\t' }).alias('csv')).select('csv.*').show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|    